__No te olvides de rellenar esto:__

- Número de grupo: 18
- Nombre de los integrantes del grupo: Hong Xiang Chen, Adrian Turiel Charro y Andrés Teruel Fernández

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de clases mediante las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q17172850' -> {'Q101436564', 'Q1067089', 'Q186506', 'Q210970', 'Q223166', ...}, )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q5994' -> {'Q3152898', 'Q4951628', 'Q52954'})

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename) as f:
    data = json.load(f)
```

Hemos optado por usar los 3 diccionarios como variables globales y crear funciones aparte que trabajen con ellas

In [1]:
import json

def load(archivo): 
    with open(archivo, encoding="utf8") as f:
        data = json.load(f)

    instrumento_etiqueta = dict()
    clase_subclase       = dict()
    clase_superclase     = dict()

    for p in data:
        a = p['x'].split(sep='/')
        b = p['y'].split(sep='/')
        instrumento_etiqueta[a[-1]] = p['xLabel']
        instrumento_etiqueta[b[-1]] = p['yLabel']
        clase_subclase.setdefault(b[-1], []).append(a[-1])
        clase_superclase.setdefault(a[-1], []).append(b[-1])

    return instrumento_etiqueta, clase_subclase, clase_superclase

* ### Comentarios:

Cargamos todos los datos en la variable data, e inicializamos los diccionarios que vamos a usar después. La forma en que añadimos los datos al diccionario consiste en: 

'a = p['x'].split(sep='/')' → Convierte algo de la forma 'http://www.wikidata.org/entity/Q11404' en un vector, donde cada elemento del vector, es el contenido entre esos '/', de forma que solo nos interesa quedarnos con el ultimo, es de decir, el 'Q11404', por eso a cada diccionario le añadimos el último elemento de ese vector (vector[-1])

-------------

In [2]:
identificador_etiqueta, subclase, superclase = load('taxonomia.json')

In [3]:
identificador_etiqueta

{'Q191464': 'zerbaghali',
 'Q11404': 'drums',
 'Q208421': 'caja',
 'Q211028': 'cacaaaaa',
 'Q221769': 'taiko',
 'Q440856': 'dhimay',
 'Q520754': 'xiangjiao gu',
 'Q537750': 'Yembe',
 'Q811075': 'batá',
 'Q849349': 'Tom-Tom',
 'Q1062015': 'janggu',
 'Q1156506': 'Dabakan',
 'Q1158124': 'Q1158124',
 'Q1196937': 'tenor drum',
 'Q1235547': 'Doli',
 'Q1321939': 'Nagara',
 'Q1436500': 'Mridanga',
 'Q1558889': 'Comunicación con tambores',
 'Q1575231': 'tambor de mano',
 'Q2118356': 'Q2118356',
 'Q2217018': 'Q2217018',
 'Q2391458': 'Q2391458',
 'Q2565221': 'Q2565221',
 'Q2754430': 'buk',
 'Q3038392': 'Dayereh',
 'Q3199707': 'Q3199707',
 'Q3623050': 'Sakara',
 'Q4450903': 'Q4450903',
 'Q4863624': 'Barril de bomba',
 'Q4905606': 'Big Drum',
 'Q5064156': 'ceremonial drum',
 'Q5211919': 'Damaha',
 'Q6458439': 'Keho',
 'Q7278496': 'Raban',
 'Q7384685': 'Maruas',
 'Q7682928': 'tanggu',
 'Q7852753': 'Tumyr',
 'Q10396699': 'Q10396699',
 'Q10829913': 'bronze drum',
 'Q11420300': 'Uchiwa daiko',
 'Q11950

In [4]:
subclase

{'Q11404': ['Q191464',
  'Q208421',
  'Q211028',
  'Q221769',
  'Q440856',
  'Q520754',
  'Q537750',
  'Q811075',
  'Q849349',
  'Q1062015',
  'Q1156506',
  'Q1158124',
  'Q1196937',
  'Q1235547',
  'Q1321939',
  'Q1436500',
  'Q1558889',
  'Q1575231',
  'Q2118356',
  'Q2217018',
  'Q2391458',
  'Q2565221',
  'Q2754430',
  'Q3038392',
  'Q3199707',
  'Q3623050',
  'Q4450903',
  'Q4863624',
  'Q4905606',
  'Q5064156',
  'Q5211919',
  'Q6458439',
  'Q7278496',
  'Q7384685',
  'Q7682928',
  'Q7852753',
  'Q10396699',
  'Q10829913',
  'Q11420300',
  'Q11950748',
  'Q12698682',
  'Q12821705',
  'Q13025534',
  'Q14640031',
  'Q15614634',
  'Q28657984',
  'Q28944914',
  'Q60617211',
  'Q60742599',
  'Q80862258',
  'Q101968884'],
 'Q12639': ['Q129321',
  'Q133722',
  'Q339801',
  'Q390110',
  'Q398619',
  'Q835684',
  'Q859236',
  'Q871633',
  'Q936762',
  'Q943438',
  'Q963334',
  'Q976406',
  'Q978694',
  'Q1014229',
  'Q1043002',
  'Q1050106',
  'Q1075560',
  'Q1195473',
  'Q1196263',
  'Q1

In [5]:
superclase
#Hijo: padres

{'Q191464': ['Q11404'],
 'Q208421': ['Q11404', 'Q55723221'],
 'Q211028': ['Q11404'],
 'Q221769': ['Q11404'],
 'Q440856': ['Q11404'],
 'Q520754': ['Q11404'],
 'Q537750': ['Q11404', 'Q212892'],
 'Q811075': ['Q11404'],
 'Q849349': ['Q11404'],
 'Q1062015': ['Q11404', 'Q5913432'],
 'Q1156506': ['Q11404'],
 'Q1158124': ['Q11404'],
 'Q1196937': ['Q11404', 'Q5199281'],
 'Q1235547': ['Q11404'],
 'Q1321939': ['Q11404'],
 'Q1436500': ['Q11404', 'Q55723287'],
 'Q1558889': ['Q11404'],
 'Q1575231': ['Q11404'],
 'Q2118356': ['Q11404'],
 'Q2217018': ['Q11404'],
 'Q2391458': ['Q11404'],
 'Q2565221': ['Q11404'],
 'Q2754430': ['Q11404'],
 'Q3038392': ['Q11404', 'Q212892'],
 'Q3199707': ['Q11404'],
 'Q3623050': ['Q11404', 'Q133163'],
 'Q4450903': ['Q11404'],
 'Q4863624': ['Q11404'],
 'Q4905606': ['Q11404'],
 'Q5064156': ['Q11404'],
 'Q5211919': ['Q11404'],
 'Q6458439': ['Q11404'],
 'Q7278496': ['Q11404'],
 'Q7384685': ['Q11404'],
 'Q7682928': ['Q11404', 'Q212892'],
 'Q7852753': ['Q11404'],
 'Q10396699': [

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [6]:
def print_tree(concepto, profundidad):

    if profundidad == 0:
        return None

    #imprimir el árbol partiendo de la profundidad 0 y el concepto 'concepto'
    print_tree_aux(concepto, profundidad , 0)


def print_tree_aux(concepto, profundidad_ini, profundidad):

    if (profundidad == profundidad_ini):
        return None

    #imprime el número de espacios necesarios según la altura del árbol
    for i in range (profundidad):
            print('   ', end='')

    print(profundidad, identificador_etiqueta[concepto], '(' + concepto + ')')

    #si el concepto no tiene hijos simplemente finaliza la función
    if not(concepto in subclase.keys()):
        return None

    #llamar a print_tree_aux para cada hijo de 'concepto'
    for c in subclase[concepto]:
        print_tree_aux(c, profundidad_ini, profundidad + 1)

* #### Comentarios:

Print_tree() es una función que hace uso de la función auxiliar print_tree_aux(). 

Print_tree_aux() es una función que se dedica a imprimir todo el árbol de forma recursiva y hace uso de un argumento extra en el que se guarda la profuncidad en la que se encuentra la función de forma que así podemos imprimirla correctamente y podemos saber cuando hemos llegado a la altura especificada.

-------------


In [7]:
print_tree('Q17172850', 3)

0 voz (Q17172850)
   1 bajo (Q27911)
      2 bajo-barítono (Q810480)
      2 bajo buffo (Q1002146)
      2 bajo profundo (Q2532487)
      2 bajo cantante (Q3636053)
      2 grave (Q5885030)
      2 octavist (Q7082656)
      2 bajo caractère (Q20638448)
      2 high bass (Q54636007)
      2 dramatic high bass (Q54636036)
      2 serious bass (Q54636068)
      2 heavy acting bass (Q54636271)
      2 Bass bourdon (Q64363543)
   1 barítono (Q31687)
      2 bajo-barítono (Q810480)
      2 character baritone (Q1062931)
      2 heldenbaritone (Q1601737)
      2 Barítono buffo (Q5721499)
      2 Barítono ligero (Q5721503)
      2 grave (Q5885030)
      2 Barítono lírico (Q8243255)
      2 Barítono dramático (Q8243257)
      2 baryton-noble (Q19740895)
      2 barítono Martin (Q21478751)
      2 Barítono martín (Q25404193)
      2 barítono verdiano (Q54635681)
      2 acting baritone (Q54635751)
      2 Kavalierbariton (Q54635784)
   1 contralto (Q37137)
      2 contralto cómica (Q5785182)
    

* #### Comentarios:

Un error que podemos encontrar en Wikidata es que hay ciertos objetos que no tienen definida su etiqueta.

Por ejemplo: '1 Q2722253 (Q2722253)'


Por otra parte, encontramos que hay algún objeto que está introducido varias veces, pero con su identificador diferente.

barítono Martin (Q21478751)
Barítono martín (Q25404193)

-------

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

In [8]:
def lcs(concepto1, concepto2):
    
    antecesores1 = antecesores(concepto1)
    antecesores2 = antecesores(concepto2)
    antecesores1.add(concepto1)
    antecesores2.add(concepto2)
    
    antecesores_comunes = antecesores1 & antecesores2
    
    aux = set()
    
    for elem in antecesores_comunes:
        aux |= antecesores(elem)
        
    antecesores_comunes -= aux
    
    return antecesores_comunes

def antecesores(concepto):

    ret = set()

    if not(concepto in superclase.keys()):
        return set()

    #añadimos todos los padres del concepto al conjunto
    for padre in superclase[concepto]:
        ret.add(padre)
        ret = ret | antecesores(padre)

    return ret

* #### Comentarios:

La función antecesores devuelve todo el conjunto de padres del concepto dado y de sus respectivos padres, de forma recursiva.

En la función lcs calculamos todos los antecesores comunes a ambos conceptos y los guardamos en antecesores_comunes.
Tras esto, nos deshacemos de todos antecesores que sean menos específicos para quedarnos al final con los antecesores comunes más bajos.

-------------

In [9]:
lcs('Q6012297', 'Q54634726')

{'Q186506'}

In [10]:
lcs('Q5885030', 'Q6012300')

{'Q17172850'}

In [11]:
lcs('Q27914', 'Q27914')

{'Q27914'}

In [12]:
lcs('Q15336282', 'Q64166304')

{'Q105738', 'Q1798603', 'Q55724840'}

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [13]:
#imprimeEtiqueta le pasamos una lista para que imprima su etiqueta con su identificador con el formato adecuado
def imprimeEtiqueta(elmens):
    
    print('[', end='')
    for x in range (len(elmens)):
    
        print(identificador_etiqueta[elmens[x]]+ ' (' + elmens[x] +')', end ='')
        
        #Controlar el ultimo elemento
        if(x < len(elmens)-1):
            print(', ', end='')
        
    print(']')

In [14]:
#Path nos devuelve el camino minimo entre el concepto1 y el concepto2
def path(concepto1,concepto2):

    ret = list()
    
    #Caso base
    if concepto1 == concepto2:
        ret.append(concepto1)
        return ret
    
    #Si concepto1 no tiene hijos, se termina devolviendo la lista.
    if not(concepto1 in subclase.keys()):
        return list()

    #Este for son las llamadas recursivas de una busqueda en profundidad
    for c in subclase[concepto1]:
        #si ret es vacio, entonces sigfica que encontramos el camino y no hace falta seguir expandiendo el arbol de busqueda
        if ret == list():
            ret = path(c, concepto2)

    if ret == list():
        return ret
    else:
        ret.append(concepto1)
        return ret

* ### Comentarios:

Para buscar el camino mas corto entre dos conceptos, usamos la función Path que recorre la taxonomía recursivamente hasta encontrarlo, para ello, miramos cada uno de los hijos del primer concepto.

---------

In [15]:
imprimeEtiqueta(path('Q186506', 'Q54634726'))

[mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)]


In [16]:
imprimeEtiqueta(path('Q17172850', 'Q6012300'))

[mezzosoprano ligera (Q6012300), alto (Q6983813), voz (Q17172850)]


In [17]:
imprimeEtiqueta(path('Q27914', 'Q27914'))

[tenor (Q27914)]


In [18]:
imprimeEtiqueta(path('Q34379', 'Q105738'))

[electrófono (Q105738), instrumento musical (Q34379)]


### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [19]:
def similarity(obj1, obj2):

    raiz = 'Q34379'

    min_comun = lcs(obj1, obj2)

    max_similarity = 0

    while min_comun != set():

        lcs_aux = min_comun.pop()

        num_aux, camino_raiz_aux, ruta1_aux, ruta2_aux = similarity_aux(obj1,obj2,lcs_aux)

        if num_aux >= max_similarity:
            max_similarity = num_aux
            max_lcs = lcs_aux
            camino_raiz = camino_raiz_aux
            ruta1 = ruta1_aux
            ruta2 = ruta2_aux

    print("Similarity: ", max_similarity)
    print('LCS: ', identificador_etiqueta[max_lcs], max_lcs)
    print('Camino a raiz: ',end = '')
    imprimeEtiqueta(camino_raiz)
    print('Ruta 1: ',end = '')
    imprimeEtiqueta(ruta1)
    print('Ruta 2: ',end = '')
    imprimeEtiqueta(ruta2)



def similarity_aux(obj1,obj2,lcs):

    raiz = 'Q34379'

    min_comun = lcs

    camino_raiz = path(raiz, min_comun)

    ruta1 = path(min_comun, obj1)

    ruta2 = path(min_comun, obj2)

    #hay que restarle -1 a las longitudes de los caminos porque hay que contar las aristas 
    return (len(camino_raiz) - 1)/(((len(camino_raiz) - 1)+(len(ruta1) - 1)+(len(ruta2) - 1))), camino_raiz, ruta1, ruta2

* ### Comentarios:

Para calcular el LCS común a los dos conceptos, reutilizamos la función LCS. Y para calcular los caminos desde la raíz al LCS y desde el LCS a cada uno de los dos conceptos, reutilizamos la función path. 

Como podemos tener varios conceptos a una misma distancia mínima, mediante el while del código y la función auxiliar, calculamos para cada uno de esos conceptos la similitud correspondiente, y guardamos aquella que es máxima.

-------

In [20]:
similarity('Q6012297', 'Q54634726')

Similarity:  0.5
LCS:  mezzosoprano Q186506
Camino a raiz: [mezzosoprano (Q186506), voz (Q17172850), instrumento musical (Q34379)]
Ruta 1: [mezzosoprano dramática (Q6012297), mezzosoprano (Q186506)]
Ruta 2: [mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)]


In [21]:
similarity('Q186506', 'Q54634726')

Similarity:  0.6666666666666666
LCS:  mezzosoprano Q186506
Camino a raiz: [mezzosoprano (Q186506), voz (Q17172850), instrumento musical (Q34379)]
Ruta 1: [mezzosoprano (Q186506)]
Ruta 2: [mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)]


In [22]:
similarity('Q27914', 'Q27914')

Similarity:  1.0
LCS:  tenor Q27914
Camino a raiz: [tenor (Q27914), high voice (Q98116969), voz (Q17172850), instrumento musical (Q34379)]
Ruta 1: [tenor (Q27914)]
Ruta 2: [tenor (Q27914)]


In [23]:
similarity('Q76239', 'Q78987')

Similarity:  0.42857142857142855
LCS:  instrumento de cuerda pulsada Q230262
Camino a raiz: [instrumento de cuerda pulsada (Q230262), instrumento de cuerda (Q1798603), cordófono (Q1051772), instrumento musical (Q34379)]
Ruta 1: [cítara (Q76239), instrumento de cuerda pulsada (Q230262)]
Ruta 2: [guitarra eléctrica (Q78987), guitarra (Q6607), plucked necked box lutes (Q57306162), instrumento de cuerda pulsada (Q230262)]


### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [24]:
def similitud_2_a_2(conjunto):

    for x in range (len(conjunto)):
        for y in range(x + 1, len(conjunto)):
            #Para mostrar los elementos que estamos comparando
            print('Comparacion: ' + identificador_etiqueta[conjunto[x]] + ', ' + identificador_etiqueta[conjunto[y]])
            #Motrar la similitud
            similarity(conjunto[x], conjunto[y])
            print(end='\n\n')

In [25]:
conjunto = ['Q5994','Q6607','Q78987','Q11405','Q8338']

In [26]:
similitud_2_a_2(conjunto)

Comparacion: piano, guitarra
Similarity:  0.2727272727272727
LCS:  instrumento de cuerda pulsada Q230262
Camino a raiz: [instrumento de cuerda pulsada (Q230262), instrumento de cuerda (Q1798603), cordófono (Q1051772), instrumento musical (Q34379)]
Ruta 1: [piano (Q5994), true board zithers with resonator box (Q4951628), true board zithers with resonator (Q55724742), true board zithers (Q55724736), board zithers (Q50829016), cítara (Q76239), instrumento de cuerda pulsada (Q230262)]
Ruta 2: [guitarra (Q6607), plucked necked box lutes (Q57306162), instrumento de cuerda pulsada (Q230262)]


Comparacion: piano, guitarra eléctrica
Similarity:  0.25
LCS:  instrumento de cuerda pulsada Q230262
Camino a raiz: [instrumento de cuerda pulsada (Q230262), instrumento de cuerda (Q1798603), cordófono (Q1051772), instrumento musical (Q34379)]
Ruta 1: [piano (Q5994), true board zithers with resonator box (Q4951628), true board zithers with resonator (Q55724742), true board zithers (Q55724736), board zit

* #### Comentarios:
Tras evaluar la salida, estamos de acuerdo con el resultado obtenido. Entre los instrumentos de las mismas familias, ya sea cuerda o viento, hay un mayor porcentaje de similitud. 

Comparando entre el piano y las guitarras (0.27%), y la flauta y la trompeta (0.4),  aunque ambas comparaciones solo tienen en común que son instrumentos de la misma familia, hay menor similitud entre el piano y las guitarras porque al tener la misma longitud del LCS hasta la raíz que la otra comparación, y mayor longitud de ruta desde los instrumentos hasta el LCS, aplicando la fórmula de similitud, obtenemos un menor porcentaje.

Entre flauta y trompeta obtenemos 0.4 porque ambos son de viento, pero no hay más similitudes ya que uno pertenece a la familia de viento metal y el otro no. Y entre el piano y las guitarras solo hay en común que son de cuerda. En cambio, entre guitarra y guitarra eléctrica se aprecia que tienen bastante en común: [guitarra (Q6607), plucked necked box lutes (Q57306162), instrumento de cuerda pulsada (Q230262), instrumento de cuerda (Q1798603), cordófono (Q1051772), instrumento musical (Q34379)].

----------------